In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import random
from sklearn.utils import shuffle

In [8]:
def check_correct(predict, y):
    result = {}
    result['cancer-correct'] = 0
    result['cancer-wrong'] = 0
    result['normal-correct'] = 0
    result['normal-wrong'] = 0

    for i in range(len(predict)) :
        if predict[i] == y[i] :
            if y[i] == 0 :
                result['normal-correct'] += 1
            else :
                result['cancer-correct'] += 1
        else :
            if y[i] == 0 :
                result['normal-wrong'] += 1
            else :
                result['cancer-wrong'] += 1

    #for result_k, result_v in result.items():
        #print(result_k +" : "+ str(result_v))
    sensitivity=result['cancer-correct']/(result['cancer-correct']+result['cancer-wrong'])
    specificity=result['normal-correct']/(result['normal-correct']+result['normal-wrong'])
    #print("Sensitivity :", sensitivity)
    #print("Specificity :", specificity)
    return sensitivity, specificity

In [58]:
path = "C:/test/coupled/"
TCGA_1 = pd.read_csv(path+"TCGA_1.csv")
TCGA_0 = pd.read_csv(path+"TCGA_0.csv")
GTEx = pd.read_csv(path+"GTEx.csv")
TCGA_1 = TCGA_1.sample(frac=1)
tr_TCGA_1 = TCGA_1[0:int(TCGA_1.shape[0]/2)]
ts_TCGA_1 = TCGA_1[int(TCGA_1.shape[0]/2):]
train_data = pd.concat([tr_TCGA_1, GTEx])
test_data = pd.concat([ts_TCGA_1, TCGA_0])
train_data = train_data.sample(frac=1)
test_data = test_data.sample(frac=1)

tr_x = train_data.drop(["patient","cancer_code","result"],axis=1)
tr_y = train_data.result
ts_x = test_data.drop(["patient","cancer_code","result"],axis=1)
ts_y = test_data.result
ts_x_n = TCGA_0.drop(["patient","cancer_code","result"],axis=1)
ts_y_n = TCGA_0.result
tr_x,tr_y = shuffle(tr_x, tr_y, random_state = 0)
ts_x,ts_y = shuffle(ts_x, ts_y, random_state = 0)
ts_x_n,ts_y_n = shuffle(ts_x_n, ts_y_n, random_state = 0)

In [59]:
from keras.layers import Input, Dense
from keras.models import Model

input_m1 = Input(shape=(308,))
h1_m1 = Dense(200,activation='relu')(input_m1)
h2_m1 = Dense(150,activation='relu')(h1_m1) 
h3_m1 = Dense(100,activation='relu')(h2_m1) 
h4_m1 = Dense(10,activation='relu')(h3_m1) 
output_m1 = Dense(1,activation="sigmoid")(h4_m1) 
model1 = Model(inputs=input_m1,outputs=output_m1)

In [60]:
print(tr_x.shape)
print(tr_y.shape)
print(ts_x.shape)
print(ts_y.shape)

(11517, 308)
(11517,)
(4988, 308)
(4988,)


In [61]:
model1.compile(optimizer='Adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])
model1.fit(tr_x,  tr_y, epochs=10)

#model1.save('my_model.h5')

Epoch 1/10
11517/11517 [==============================] - 3s 261us/step - loss: 0.0366 - acc: 0.9848
Epoch 2/10
11517/11517 [==============================] - 2s 159us/step - loss: 0.0072 - acc: 0.9980
Epoch 3/10
11517/11517 [==============================] - 2s 156us/step - loss: 0.0044 - acc: 0.9985
Epoch 4/10
11517/11517 [==============================] - 2s 158us/step - loss: 0.0026 - acc: 0.9990
Epoch 5/10
11517/11517 [==============================] - 2s 157us/step - loss: 0.0056 - acc: 0.9977
Epoch 6/10
11517/11517 [==============================] - 2s 160us/step - loss: 0.0035 - acc: 0.9990
Epoch 7/10
11517/11517 [==============================] - 2s 159us/step - loss: 0.0023 - acc: 0.9994
Epoch 8/10
11517/11517 [==============================] - 2s 157us/step - loss: 0.0084 - acc: 0.9971
Epoch 9/10
11517/11517 [==============================] - 2s 158us/step - loss: 3.3775e-05 - acc: 1.0000
Epoch 10/10
11517/11517 [==============================] - 2s 156us/step - loss: 3.0938

In [63]:
print(model1.evaluate(tr_x, tr_y))
print(model1.evaluate(ts_x, ts_y))
print(model1.evaluate(ts_x_n, ts_y_n))

11517/11517 [==============================] - 1s 73us/step
[1.4365177510860448e-06, 1.0]
4988/4988 [==============================] - 0s 52us/step
[1.5798518948384703, 0.8757016840894986]
656/656 [==============================] - 0s 76us/step
[11.9380692040048, 0.06554878048780488]
